## 📦 1. Installation et Setup

Installation des dépendances nécessaires

In [ ]:
# Installation des packages requis
!pip install -q torch torchvision gymnasium pygame numpy matplotlib seaborn pyyaml tqdm scipy pandas tensorboard

In [ ]:
# Vérifier GPU disponible
import torch
print(f"🔥 CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  CPU mode (sera plus lent)")

## 📁 2. Cloner le Projet

Clone ton repository GitHub

In [ ]:
# Cloner le projet depuis GitHub
!git clone https://github.com/mohhajji-1111/projet_RL.git
%cd projet_RL

print("✅ Projet cloné avec succès!")

In [ ]:
# Vérifier la structure du projet
!ls -la

## 🔧 3. Configuration de l'Environnement

Setup de l'environnement Python

⚠️ **Important**: Cette cellule configure le Python path pour que les imports fonctionnent correctement.

---

In [ ]:
import sys
import os
from pathlib import Path

# Ajouter le projet au Python path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Vérifier que les modules sont accessibles
print(f"✅ Project root: {project_root}")
print(f"✅ Python path configuré")
print(f"\n📁 Structure du projet:")
!ls -la src/
!ls -la src/agents/

## 🎮 4. Test de l'Environnement

Vérifier que l'environnement fonctionne

In [ ]:
from src.environment.navigation_env import NavigationEnv
import numpy as np

# Créer environnement
env = NavigationEnv(width=800, height=600, render_mode=None)

print(f"✅ Environnement créé avec succès!")
print(f"   Observation space: {env.observation_space.shape}")
print(f"   Action space: {env.action_space.n} actions")

# Test rapide
state, _ = env.reset()
print(f"\n📊 État initial: {state}")

# Faire quelques steps
for i in range(5):
    action = env.action_space.sample()
    state, reward, done, truncated, info = env.step(action)
    print(f"   Step {i+1}: action={action}, reward={reward:.2f}, done={done}")

print("\n✅ Environnement fonctionne correctement!")

## 🧠 5. Initialiser l'Agent avec Curiosité

Créer l'agent CuriosityAgent avec le module ICM

In [ ]:
from src.agents.curiosity_agent import CuriosityAgent
import yaml

# Charger la configuration
with open('configs/curiosity_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Créer l'agent
device = 'cuda' if torch.cuda.is_available() else 'cpu'

agent = CuriosityAgent(
    state_dim=env.observation_space.shape[0],
    action_dim=env.action_space.n,
    config=config['agent'],
    device=device
)

print(f"✅ Agent créé avec succès!")
print(f"   Device: {device}")
print(f"   State dim: {env.observation_space.shape[0]}")
print(f"   Action dim: {env.action_space.n}")
print(f"   Feature dim: {config['agent']['feature_dim']}")
print(f"   Curiosity beta: {config['agent']['curiosity_beta']}")

# Afficher les réseaux
print("\n📊 Architecture des réseaux ICM:")
print(f"   Feature Network: {sum(p.numel() for p in agent.feature_net.parameters())} params")
print(f"   Inverse Model: {sum(p.numel() for p in agent.inverse_model.parameters())} params")
print(f"   Forward Model: {sum(p.numel() for p in agent.forward_model.parameters())} params")
print(f"   Q-Network: {sum(p.numel() for p in agent.q_network.parameters())} params")

## 🚀 6. Entraînement de l'Agent

Lancer l'entraînement complet avec suivi en temps réel

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import deque

# Paramètres d'entraînement
num_episodes = 1500
eval_interval = 50
save_interval = 100

# Créer dossiers pour sauvegardes
os.makedirs('results/models/curiosity', exist_ok=True)
os.makedirs('results/plots', exist_ok=True)
os.makedirs('results/logs', exist_ok=True)

# Métriques
episode_rewards = []
episode_lengths = []
intrinsic_rewards = []
success_rate = deque(maxlen=100)
avg_rewards = deque(maxlen=100)

# ICM losses
forward_losses = []
inverse_losses = []

best_reward = -float('inf')

print("🚀 Début de l'entraînement...\n")

# Barre de progression
pbar = tqdm(range(num_episodes), desc="Training")

for episode in pbar:
    state, _ = env.reset()
    episode_reward = 0
    episode_intrinsic = 0
    episode_length = 0
    done = False
    truncated = False
    
    while not (done or truncated):
        # Sélectionner action
        action = agent.select_action(state)
        
        # Step dans l'environnement
        next_state, reward, done, truncated, info = env.step(action)
        
        # Calculer récompense intrinsèque
        intrinsic_reward = agent.compute_intrinsic_reward(
            torch.FloatTensor(state).unsqueeze(0).to(device),
            torch.LongTensor([action]).to(device),
            torch.FloatTensor(next_state).unsqueeze(0).to(device)
        )
        
        # Stocker transition
        agent.store_transition(state, action, reward, next_state, done or truncated)
        
        # Entraîner
        if len(agent.replay_buffer) > agent.batch_size:
            loss = agent.train_step()
        
        episode_reward += reward
        episode_intrinsic += intrinsic_reward
        episode_length += 1
        state = next_state
    
    # Enregistrer métriques
    episode_rewards.append(episode_reward)
    episode_lengths.append(episode_length)
    intrinsic_rewards.append(episode_intrinsic)
    avg_rewards.append(episode_reward)
    success_rate.append(1.0 if info.get('success', False) else 0.0)
    
    # ICM stats
    icm_stats = agent.get_icm_stats()
    forward_losses.append(icm_stats['forward_loss'])
    inverse_losses.append(icm_stats['inverse_loss'])
    
    # Update progress bar
    pbar.set_postfix({
        'Reward': f"{episode_reward:.2f}",
        'Avg': f"{np.mean(avg_rewards):.2f}",
        'Success': f"{np.mean(success_rate):.2%}",
        'Epsilon': f"{agent.epsilon:.3f}"
    })
    
    # Sauvegarder meilleur modèle
    if episode_reward > best_reward:
        best_reward = episode_reward
        agent.save_checkpoint('results/models/curiosity/best.pth')
    
    # Sauvegardes régulières
    if (episode + 1) % save_interval == 0:
        agent.save_checkpoint(f'results/models/curiosity/checkpoint_{episode+1}.pth')
        print(f"\n💾 Checkpoint sauvegardé: episode {episode+1}")

print("\n✅ Entraînement terminé!")
print(f"   Meilleure récompense: {best_reward:.2f}")
print(f"   Taux de succès final: {np.mean(success_rate):.2%}")
print(f"   Récompense moyenne (100 derniers): {np.mean(avg_rewards):.2f}")

## 📊 7. Visualisation des Résultats

Analyse des performances de l'agent

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter1d

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 12)

fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# 1. Récompenses d'épisode
ax = axes[0, 0]
ax.plot(episode_rewards, alpha=0.3, label='Raw', color='blue')
smoothed = gaussian_filter1d(episode_rewards, sigma=20)
ax.plot(smoothed, label='Smoothed (σ=20)', color='red', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Total Reward')
ax.set_title('📈 Récompenses d\'Épisode')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Moyenne mobile
ax = axes[0, 1]
window = 100
moving_avg = [np.mean(episode_rewards[max(0, i-window):i+1]) for i in range(len(episode_rewards))]
ax.plot(moving_avg, color='green', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Average Reward')
ax.set_title(f'📊 Moyenne Mobile (window={window})')
ax.grid(True, alpha=0.3)

# 3. Récompenses intrinsèques
ax = axes[1, 0]
ax.plot(intrinsic_rewards, alpha=0.4, color='orange')
smoothed_int = gaussian_filter1d(intrinsic_rewards, sigma=20)
ax.plot(smoothed_int, color='red', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Intrinsic Reward')
ax.set_title('🔍 Récompenses Intrinsèques (Curiosité)')
ax.grid(True, alpha=0.3)

# 4. ICM Losses
ax = axes[1, 1]
ax.plot(gaussian_filter1d(forward_losses, sigma=20), label='Forward Loss', color='purple', linewidth=2)
ax.plot(gaussian_filter1d(inverse_losses, sigma=20), label='Inverse Loss', color='brown', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Loss')
ax.set_title('🧠 ICM Losses')
ax.legend()
ax.grid(True, alpha=0.3)

# 5. Longueur des épisodes
ax = axes[2, 0]
ax.plot(episode_lengths, alpha=0.3, color='cyan')
smoothed_len = gaussian_filter1d(episode_lengths, sigma=20)
ax.plot(smoothed_len, color='darkblue', linewidth=2)
ax.set_xlabel('Episode')
ax.set_ylabel('Steps')
ax.set_title('⏱️ Longueur des Épisodes')
ax.grid(True, alpha=0.3)

# 6. Distribution des récompenses
ax = axes[2, 1]
ax.hist(episode_rewards, bins=50, alpha=0.7, color='green', edgecolor='black')
ax.axvline(np.mean(episode_rewards), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(episode_rewards):.2f}')
ax.axvline(np.median(episode_rewards), color='blue', linestyle='--', linewidth=2, label=f'Median: {np.median(episode_rewards):.2f}')
ax.set_xlabel('Reward')
ax.set_ylabel('Frequency')
ax.set_title('📊 Distribution des Récompenses')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/plots/training_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualisations sauvegardées dans results/plots/")

## 📈 8. Statistiques Détaillées

In [ ]:
# Statistiques finales
print("="*60)
print("📊 STATISTIQUES FINALES")
print("="*60)

print(f"\n🎯 Récompenses:")
print(f"   Moyenne totale: {np.mean(episode_rewards):.2f} ± {np.std(episode_rewards):.2f}")
print(f"   Médiane: {np.median(episode_rewards):.2f}")
print(f"   Minimum: {np.min(episode_rewards):.2f}")
print(f"   Maximum: {np.max(episode_rewards):.2f}")
print(f"   Meilleur: {best_reward:.2f}")

# Derniers 100 épisodes
last_100 = episode_rewards[-100:]
print(f"\n📊 Performance (100 derniers épisodes):")
print(f"   Moyenne: {np.mean(last_100):.2f} ± {np.std(last_100):.2f}")
print(f"   Taux de succès: {np.mean(list(success_rate)):.2%}")

print(f"\n⏱️ Épisodes:")
print(f"   Longueur moyenne: {np.mean(episode_lengths):.1f} steps")
print(f"   Longueur médiane: {np.median(episode_lengths):.1f} steps")

print(f"\n🔍 Curiosité (ICM):")
print(f"   Récompense intrinsèque moyenne: {np.mean(intrinsic_rewards):.4f}")
print(f"   Forward loss finale: {forward_losses[-1]:.4f}")
print(f"   Inverse loss finale: {inverse_losses[-1]:.4f}")

print(f"\n🧠 Agent:")
print(f"   Epsilon final: {agent.epsilon:.4f}")
print(f"   Replay buffer size: {len(agent.replay_buffer)}")

print("\n" + "="*60)

## 💾 9. Télécharger les Modèles

Télécharger les modèles entraînés vers Google Drive

In [ ]:
# Monter Google Drive (optionnel)
from google.colab import drive
drive.mount('/content/drive')

# Copier les résultats vers Drive
!cp -r results /content/drive/MyDrive/projet_RL_results

print("✅ Résultats copiés vers Google Drive: MyDrive/projet_RL_results/")

In [ ]:
# Ou télécharger directement
from google.colab import files

# Compresser les résultats
!zip -r results.zip results/

# Télécharger
files.download('results.zip')

print("✅ Archive results.zip téléchargée!")

## 🎮 10. Test de l'Agent Entraîné

Tester l'agent sur quelques épisodes

In [ ]:
# Charger le meilleur modèle
agent.load_checkpoint('results/models/curiosity/best.pth')
agent.epsilon = 0.0  # Mode exploitation pur

print("🎮 Test de l'agent entraîné (10 épisodes)...\n")

test_rewards = []
test_lengths = []

for ep in range(10):
    state, _ = env.reset()
    episode_reward = 0
    steps = 0
    done = False
    truncated = False
    
    while not (done or truncated) and steps < 500:
        action = agent.select_action(state)
        state, reward, done, truncated, info = env.step(action)
        episode_reward += reward
        steps += 1
    
    test_rewards.append(episode_reward)
    test_lengths.append(steps)
    success = "✅" if info.get('success', False) else "❌"
    print(f"Episode {ep+1}: {success} Reward={episode_reward:.2f}, Steps={steps}")

print(f"\n📊 Résultats du test:")
print(f"   Moyenne: {np.mean(test_rewards):.2f} ± {np.std(test_rewards):.2f}")
print(f"   Médiane: {np.median(test_rewards):.2f}")
print(f"   Longueur moyenne: {np.mean(test_lengths):.1f} steps")

## 📝 11. Sauvegarder le Rapport Final

In [ ]:
# Créer un rapport texte
report = f"""
RAPPORT D'ENTRAÎNEMENT - CURIOSITY AGENT
{'='*60}

Configuration:
- Épisodes: {num_episodes}
- Device: {device}
- Feature dim: {config['agent']['feature_dim']}
- Curiosity beta: {config['agent']['curiosity_beta']}

Résultats:
- Meilleure récompense: {best_reward:.2f}
- Récompense moyenne: {np.mean(episode_rewards):.2f} ± {np.std(episode_rewards):.2f}
- Récompense médiane: {np.median(episode_rewards):.2f}
- Taux de succès (final): {np.mean(list(success_rate)):.2%}

Performance (100 derniers épisodes):
- Récompense moyenne: {np.mean(last_100):.2f} ± {np.std(last_100):.2f}
- Longueur moyenne: {np.mean(episode_lengths[-100:]):.1f} steps

Curiosité (ICM):
- Récompense intrinsèque moyenne: {np.mean(intrinsic_rewards):.4f}
- Forward loss finale: {forward_losses[-1]:.4f}
- Inverse loss finale: {inverse_losses[-1]:.4f}

Test (10 épisodes):
- Moyenne: {np.mean(test_rewards):.2f} ± {np.std(test_rewards):.2f}
- Médiane: {np.median(test_rewards):.2f}
- Longueur moyenne: {np.mean(test_lengths):.1f} steps

{'='*60}
"""

# Sauvegarder
with open('results/training_report.txt', 'w') as f:
    f.write(report)

print(report)
print("✅ Rapport sauvegardé: results/training_report.txt")

## 🎉 Fin de l'Entraînement!

### Fichiers générés:
- `results/models/curiosity/best.pth` - Meilleur modèle
- `results/models/curiosity/checkpoint_*.pth` - Checkpoints
- `results/plots/training_summary.png` - Visualisations
- `results/training_report.txt` - Rapport détaillé

### Prochaines étapes:
1. Télécharger les modèles entraînés
2. Tester localement avec l'interface GUI
3. Comparer avec DQN baseline
4. Optimiser les hyperparamètres si nécessaire

**Bon courage! 🚀🤖**